In [1]:
! python -m pip install numpy matplotlib pandas scipy seaborn scikit-learn tensorflow keras_tuner google-genai pypdf

  Using cached numpy-2.4.1-cp313-cp313-win_amd64.whl.metadata (6.6 kB)
  Using cached matplotlib-3.10.8-cp313-cp313-win_amd64.whl.metadata (52 kB)
  Using cached pandas-3.0.0-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached scipy-1.17.0-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.8.0-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached keras_tuner-1.4.8-py3-none-any.whl.metadata (5.6 kB)
  Using cached google_genai-1.60.0-py3-none-any.whl.metadata (53 kB)
  Using cached pypdf-6.6.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp313-cp313-win_amd64.whl.metadata (116 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-win_amd64.whl.metadata (6.4 kB)
  


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from google import genai
from pypdf import PdfReader
import numpy as np

In [ ]:
api_key = ''
client = genai.Client(api_key=api_key)

SpaceBot - Ask me anything.

I den här delen av kunskapskontrollen ska jag skapa en chattbot med RAG. All fakta och information som chattbotten kommer använda kommer från en PDF-fil från NASA om vårt solsystem. Chattbotten ska kunna svara på allt från enkla frågor till mer detaljerade frågor om våra planeter, stjärnor och mycket mer.

I nedanstående kod läser jag in PDF-filen som chattbotten använder information om.

In [20]:
reader = PdfReader("pdf_file/solarsystem.pdf")

text = ""

for page in reader.pages:
    text += page.extract_text()

Chunks

In [21]:
chunks = []
n = 800
overlap = 150

for i in range(0, len(text), n - overlap):
    chunks.append(text[i:i + n])

print(f"Amount of chunks: {len(chunks)}.")

Amount of chunks: 224.


In [22]:
for chunk in chunks:
    print(chunk)
    print("----")

National Aeronautics and Space AdministrationNational Aeronautics and Space Administration
OURSOLARSYSTEM
2013
www.nasa.gov
Inside
Our Solar System
Our Star — The Sun
Mercury
Venus
Earth
Earth’s Moon
Mars
Asteroids
Meteors and Meteorites 
Moons of the Solar System
Jupiter
Galilean Moons of Jupiter
Saturn
Moons of Saturn
Uranus
Neptune
Pluto and Charon
Comets
Kuiper Belt and Oort Cloud
What Is a Planet?
Educational Product
Educators Grades K–12+
LS-2013-07-003-HQ 
JPL 400-1489  07/13NASA EDUCATIONAL RESOURCES
The NASA portal (www.nasa.gov) is the gateway for information 
about content, programs, and services offered for the general 
public and the education community. NASA’s goal is to improve 
interactions for students, educators, and families with NASA 
and its education resources. 
NASA’
----
 the education community. NASA’s goal is to improve 
interactions for students, educators, and families with NASA 
and its education resources. 
NASA’s education home page (www.nasa.gov; click o

In [23]:
print(chunks[4])

rces, directions, 
hours of operation, and other information. 
Go to www.nasa.gov and click on “For Educators” to locate the 
Regional ERCs.
NASA Wavelength (www.nasawavelength.org) is a digital col-
lection of Earth and space science resources for educators of all 
levels, from elementary to college, to out-of-school programs. 
The resources were developed through funding from the NASA 
Science Mission Directorate and have been peer-reviewed by 
educators and scientists.
LS-2013-07-003-HQ — JPL 400-1489A  07/13National Aeronautics and Space Administration
National Aeronautics and Space Administration
Mercury
Venus
Earth
Mars
Jupiter
Saturn
Uranus Neptune
Our Solar System
www.nasa.govHumans have gazed at the heavens and tried to understand 
the cosmos for thousands of years. Ancient civili


In [24]:
chunks_clean = []

for chunk in chunks:
    if(
       len(chunk) > 400
       and chunk.count(".") >= 2
       and "www.nasa.gov" not in chunk
       and "OUR SOLAR SYSTEM" not in chunk.upper()

    ):
        chunks_clean.append(chunk)
    
print(f"Amount of chunks after cleaning: {len(chunks_clean)}")

Amount of chunks after cleaning: 184


In [25]:
for i, chunk in enumerate(chunks_clean[:3]):
    print(f"\n--- Clean Chunk {i+1} ---\n")
    print(chunk)


--- Clean Chunk 1 ---

n — and these are the 
names astronomers use today. Planetary features are named by 
the International Astronomical Union, founded in 1919. For more 
information about the names of planets, moons, and features, 
consult the Gazetteer of Planetary Nomenclature website at 
planetarynames.wr.usgs.gov.
Ancient observers believed that the Sun and all the other ce-
lestial bodies revolved around Earth. Astronomers gradually 
realized that the Earth-centered model did not account for the 
motions of the planets. In the early 17th century, Galileo Gali-
lei’s discoveries using the recently invented telescope strongly 
supported the concept of a “solar system” in which all the plan-
ets, including Earth, revolve around a central star — the Sun. 
Planetary moons, the rings of Saturn, and more planet

--- Clean Chunk 2 ---

e four 
planets closest to the Sun — Mercury, Venus, Earth, and Mars — 
are called the terrestrial planets because they have solid, rocky 
surfaces. Tw

In [26]:
print(chunks_clean[4])

om 
the Sun. Scientists anticipate that Voyager 1 will cross into inter-
stellar space, where gas and dust from other stars are found as 
well as the enormous Oort Cloud, within a few months to a few 
years. Both spacecraft should have enough electrical power to 
send data until at least 2020. It will be thousands of years before 
the two Voyagers exit the Oort Cloud, a vast spherical shell of icy 
bodies surrounding the solar system.
As we explore the universe, we wonder: Are there other planets 
where life might exist? Are we alone? These are the great ques-
tions that science is now probing. Only recently have astrono-
mers had the tools — sensitive telescopes on Earth and in space 
— to detect planets orbiting stars in other solar systems. 
FAST FACTS
   
                   
   
      
